In [1]:
import json
import os
import random
import numpy as np
import pandas as pd

random.seed(42)
np.random.seed(42)

In [2]:
OUTPUT_DATA_DIR = "./output_data/"

train_df = pd.read_csv(OUTPUT_DATA_DIR+"interactions_training.csv")

val_df = pd.read_csv(OUTPUT_DATA_DIR+"interactions_validation.csv")

test_df = pd.read_csv(OUTPUT_DATA_DIR+"interactions_testing.csv")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

def process_book_text(book_text, exclude_text, ps):
    """Pre-processes the text given by `review_text`.

    Parameters
    ----------
    book_text: str
        The book text to be processed.
    exclude_text: collection
        A collection of words to be excluded.
    ps: PorterStemmer
        The PorterStemmer used to perform word stemming.

    Returns
    -------
    str
        A string representing the processed version of `review_text`.

    """
    book = re.sub('[^a-zA-Z0-9]', ' ', book_text).lower().split()
    book = [ps.stem(word) for word in book if not word in exclude_text]
    return ' '.join(book)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Matthew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def preprocess_for_classification(data_df):
    """Preprocesses `data_df` to be used in classification.

    Parameters
    ----------
    data_df: pd.DataFrame
        The DataFrame to be processed.

    Returns
    -------
    pd.DataFrame
        The DataFrame obtained from `data_df` after processing.

    """
    # flags for most popular formats
    data_df['format'] = data_df['format'].apply(lambda x: str(x).lower())
    data_df['is_paperback'] = data_df['format'].apply(lambda x: int("paper" in x))
    data_df['is_hardcover'] = data_df['format'].apply(lambda x: int("hard" in x))
    data_df['is_audio'] = data_df['format'].apply(lambda x: int("audio" in x))
    data_df['is_other_format'] = (data_df['is_paperback'] + data_df['is_hardcover'] +
                                  data_df['is_audio'] + data_df['is_ebook'])
    data_df['is_other_format'] = data_df['is_other_format'].apply(lambda x: 0 if x > 0 else 1)

    #flags for most popular publishers
    data_df['publisher'] = data_df['publisher'].apply(lambda x: str(x).lower())
    data_df['from_penguin'] = data_df['publisher'].apply(lambda x: int("penguin" in x))
    data_df['from_harpercollins'] = data_df['publisher'].apply(lambda x: int("harpercollins" in x or "harper collins" in x))
    data_df['from_university_press'] = data_df['publisher'].apply(lambda x: int("university press" in x))
    data_df['from_vintage'] = data_df['publisher'].apply(lambda x: int("vintage" in x))
    data_df['from_createspace'] = data_df['publisher'].apply(lambda x: int("createspace" in x or "create space" in x))
    data_df['other_publisher'] = (data_df['from_penguin'] + data_df['from_harpercollins'] +
                                  data_df['from_university_press'] + data_df['from_vintage'] + data_df['from_createspace'])
    data_df['other_publisher'] = data_df['other_publisher'].apply(lambda x: 0 if x > 0 else 1)

    # ensuring columns are not missing
    data_df['average_rating'] = data_df['average_rating'].apply(lambda x: 0.0 if pd.isnull(x) else x)
    train_df['text_reviews_count'] = data_df['text_reviews_count'].apply(lambda x: 0 if pd.isnull(x) else x)
    data_df['ratings_count'] = data_df['ratings_count'].apply(lambda x: 0 if pd.isnull(x) else x)
    median_page_count = data_df['num_pages'].median()

    data_df['num_pages'] = data_df['num_pages'].apply(lambda x: median_page_count if pd.isnull(x) else x)

    # flags for most popular authors
    data_df['main_author'] = data_df['main_author'].astype(str)
    data_df['author_a'] = data_df['main_author'].apply(lambda x: int(x == "435477.0"))
    data_df['author_b'] = data_df['main_author'].apply(lambda x: int(x == "903.0"))
    data_df['author_c'] = data_df['main_author'].apply(lambda x: int(x == "947.0"))
    data_df['author_d'] = data_df['main_author'].apply(lambda x: int(x == "4624490.0"))
    data_df['author_e'] = data_df['main_author'].apply(lambda x: int(x == "18540.0"))
    data_df['author_f'] = data_df['main_author'].apply(lambda x: int(x == "8075577.0"))
    data_df['author_other'] = (data_df['author_a'] + data_df['author_b'] +
                                data_df['author_c'] + data_df['author_d'] +
                                data_df['author_e'] + data_df['author_f'])
    data_df['author_other'] = data_df['author_other'].apply(lambda x: 0 if x > 0 else 1)
    return data_df

In [5]:
def preprocess_all_book_text(data_df, id_col, text_col, exclude_text, ps):
    """Preprocesses the book text in `data_df` for `text_col`.

    The dataframe is restricted to `id_col` and `text_col` and then the
    unique ids are chosen. This is so that we only preprocess the text
    for a book once. Then we join the resulting text back to `data_df`.

    Parameters
    ----------
    data_df: pd.DataFrame
        The DataFrame containing the data to be preprocessed.
    id_col: str
        The column from which unique ids are chosen.
    text_col: str
        The column to be pre-processed.
    exclude_text: collection
        A collection of words to remove
    ps: PorterStemmer
        The PorterStemmer used for word stemming.

    Returns
    -------
    pd.DataFrame
        The DataFrame obtained from `data_df` after adding a column
        with the processed text.

    """
    book_df = data_df[[id_col, text_col]]
    book_df = book_df.drop_duplicates(subset=[id_col])
    book_df['cleaned_text'] = book_df[text_col].apply(lambda x: process_book_text(x, exclude_text, ps))
    final_df = pd.merge(data_df, book_df[[id_col, "cleaned_text"]], how="inner", on=[id_col])
    return final_df

In [6]:
def run_preprocess_pipeline(data_df, exclude_text, ps):
    """Runs the full pre-processing pipeline on `data_df`.

    Parameters
    ----------
    data_df: pd.DataFrame

    """
    processed_df = preprocess_for_classification(data_df)
    return preprocess_all_book_text(processed_df, "book_id", "title_description", exclude_text, ps)

In [7]:
exclude_english = set(stopwords.words('english'))
ps = PorterStemmer()
train_df_processed = run_preprocess_pipeline(train_df, exclude_english, ps)
val_df_processed = run_preprocess_pipeline(val_df, exclude_english, ps)
test_df_processed = run_preprocess_pipeline(test_df, exclude_english, ps)

In [8]:
train_df_processed

,user_id,book_id,review_id,is_read,rating,review_text_incomplete,date_added,date_updated,read_at,started_at,...,from_createspace,other_publisher,author_a,author_b,author_c,author_d,author_e,author_f,author_other,cleaned_text
0,55e33e664f052f3858d8df5d9cd38597,2547,a3a8cf3cead1f647c240ce56d394090d,True,5,NaN,Fri Oct 18 14:39:29 -0700 2013,Fri Oct 18 14:39:29 -0700 2013,NaN,NaN,...,0,1,0,0,0,0,0,0,1,prophet kahlil gibran masterpiec prophet one b...
1,b895142b8e9211781797fb5183db1607,2547,29ac8c961f92d37924d9baaaf82346bf,True,4,NaN,Sun Feb 02 07:44:33 -0800 2014,Sun Feb 02 07:44:33 -0800 2014,NaN,NaN,...,0,1,0,0,0,0,0,0,1,prophet kahlil gibran masterpiec prophet one b...
2,1b803e1294962dee7ba2fa786cc784e2,2547,27ba784279300ef5bbe2f6aa087324bc,True,5,NaN,Thu Aug 04 06:43:11 -0700 2011,Thu Aug 04 06:43:11 -0700 2011,NaN,NaN,...,0,1,0,0,0,0,0,0,1,prophet kahlil gibran masterpiec prophet one b...
3,e159aee0ab026edbbcfae81b36efe72c,2547,0f95079de88cde4e024a26c8896e590d,True,4,NaN,Mon Dec 08 19:37:35 -0800 2014,Mon Dec 08 19:37:35 -0800 2014,NaN,NaN,...,0,1,0,0,0,0,0,0,1,prophet kahlil gibran masterpiec prophet one b...
4,b1b585d4de9e3524c98f12554b9c56b4,2547,e904e32673dc13d8a35c53e571fcf11e,False,0,NaN,Sun Oct 22 21:43:23 -0700 2017,Sun Oct 22 21:43:23 -0700 2017,NaN,NaN,...,0,1,0,0,0,0,0,0,1,prophet kahlil gibran masterpiec prophet one b...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257640,83d4c1428afe4b0df30af8fc55bd273c,237611,31bbe3f2f4006a5c02ec013ef0099c9d,False,0,NaN,Tue Sep 15 13:14:10 -0700 2009,Tue Sep 15 13:14:10 -0700 2009,NaN,NaN,...,0,0,0,0,0,0,0,0,1,translat neruda way macchu picchu goe translat...
257641,8d78dea6e0bb399718a0a02a8e7285d4,18329474,fc4c11a914886a0389b1e2f37a5ec893,True,5,""" Even your Weakness<br />,,Can be a strength<...",Fri Sep 13 04:24:02 -0700 2013,Tue Dec 03 11:19:55 -0800 2013,Sun Nov 24 00:00:00 -0800 2013,Fri Sep 13 00:00:00 -0700 2013,...,0,1,0,0,0,0,0,0,1,word boat word thought help other simpl word c...
257642,15ab66649268d565e97f3c1c58df4563,32934749,e2f79ac4f775c948c4a051a52cd347e7,True,3,NaN,Fri Jun 02 04:55:12 -0700 2017,Fri Jun 02 04:55:12 -0700 2017,NaN,NaN,...,0,1,0,0,0,0,0,0,1,select poem emili dickinson born promin new en...
257643,1a8db358578612605b1b3dbf83a3ed75,19017083,2df89dec3f0350a8e015280cbec4c7d4,False,0,NaN,Wed Jan 29 13:15:54 -0800 2014,Wed Jan 29 13:15:54 -0800 2014,NaN,NaN,...,0,1,0,0,0,0,0,0,1,gather evid pinpoint accuraci virtuos humour c...


In [ ]:
book_texts = train_df_processed[['book_id', 'cleaned_text']]
book_texts = book_texts.drop_duplicates(subset=['book_id'])

full_text = " ".join(list(book_texts['cleaned_text'])) + " "

In [ ]:
from wordcloud import WordCloud
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('whitegrid')
plt.style.use('seaborn-darkgrid')
%matplotlib inline
sns.set(font_scale=1.5)

word_cloud = WordCloud(width=800, height=800,
                       background_color='white',
                       min_font_size=10).generate(full_text)

plt.figure(figsize=(8,8), facecolor=None)
plt.imshow(word_cloud)
plt.axis("off")
plt.tight_layout(pad=0)

plt.show()

In [ ]:
print(len(train_df_processed) + len(val_df_processed) + len(test_df_processed))

In [ ]:
import json

interactions_file = os.path.join("data", "goodreads_interactions_poetry.json")

users = []
with open(interactions_file, 'r') as data_file:
    for row in data_file:
        users.append(json.loads(row)['user_id'])

np_users = np.array(users)
del users

In [ ]:
print(len(np.unique(np_users)))

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
from sklearn.utils import shuffle

def shuffle_dataset(data_df):
    """Randomly shuffles `df`.

    Parameters
    ----------
    df: pd.DataFrame
        The DataFrame to be shuffled.

    Returns
    -------
    pd.DataFrame
        A shuffled dataframe obtained from `df`.

    """
    data_df = shuffle(data_df)
    data_df.reset_index(inplace=True, drop=True)
    return data_df

In [ ]:
train_df_processed = shuffle_dataset(train_df_processed)
val_df_processed = shuffle_dataset(val_df_processed)
test_df_processed = shuffle_dataset(test_df_processed)

In [ ]:
train_df_processed.to_csv(OUTPUT_DATA_DIR+"text_processed_training.csv", index=False)
val_df_processed.to_csv(OUTPUT_DATA_DIR+"text_processed_validation.csv", index=False)
test_df_processed.to_csv(OUTPUT_DATA_DIR+"text_processed_testing.csv", index=False)